## PROYECTO USED CAR SALES 

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Introducción.

Para desarrollar este proyecto, vamos a trabajar con los datos de la empresa Rusty Bargain, que es un servicio de venta de coches de segunda mano que está desarrollando una app para atraer a nuevos clientes. El objetivo es desarrollar un modelo que determine el valor de mercado del automóvil.

Para ello vamos a trabajar con varios modelos con ajuste de hiperparámetros:

- Modelo de regresión lineal

- Modelo de bosque aleatorio

- Modelo LightGBM

- Modelo CatBoost

Utilizaremos la métrica RECM para evaluar el desempeño de cada uno los modelos.


## Preparación de datos

Primero procedemos a importar Librerías:

In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import catboost as cb

from sklearn.impute import SimpleImputer
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV

Leemos el Conjunto de Datos con la información de la empresa Rusty Bargain:

In [2]:
vehicles = pd.read_csv("/datasets/car_data.csv")

Mostramos información general sobre el Conjunto de Datos:

In [3]:
# Imprimimos la información general/resumida del conjunto de datos.
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# Revisamos los datos faltantes en el conjunto de datos.
vehicles.isnull().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [5]:
%%time
# Convertimos las fechas a datetime
vehicles['DateCrawled'] = pd.to_datetime(vehicles['DateCrawled'], errors='coerce')
vehicles['DateCreated'] = pd.to_datetime(vehicles['DateCreated'], errors='coerce')
vehicles['LastSeen'] = pd.to_datetime(vehicles['LastSeen'], errors='coerce')

CPU times: user 1min 19s, sys: 63.7 ms, total: 1min 19s
Wall time: 1min 19s


In [6]:
# Verificamos la conversión de las columnas de fechas
vehicles.dtypes

DateCrawled          datetime64[ns]
Price                         int64
VehicleType                  object
RegistrationYear              int64
Gearbox                      object
Power                         int64
Model                        object
Mileage                       int64
RegistrationMonth             int64
FuelType                     object
Brand                        object
NotRepaired                  object
DateCreated          datetime64[ns]
NumberOfPictures              int64
PostalCode                    int64
LastSeen             datetime64[ns]
dtype: object

In [7]:
# Imprimimos las primeras líneas para ver el conjunto de datos.
vehicles.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:00,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-07-04 03:16:00
1,2016-03-24 10:58:00,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-07-04 01:46:00
2,2016-03-14 12:52:00,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14,0,90480,2016-05-04 12:47:00
3,2016-03-17 16:54:00,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:00
4,2016-03-31 17:25:00,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-06-04 10:17:00


In [8]:
# Utilizamos el método describe() para realizar una exploración rápida inicial de las variables numéricas
# del conjunto de datos.
vehicles.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


A partir del resultado obtenido del método describe(), podemos indicar los siguientes comentarios: Comentarios:

Price:
La variable objetivo muestra una distribución que va desde 0 hasta 20,000, con una media de 4,416.66 y una desviación estándar alta de 4,514.16. Esto indica una amplia variabilidad en los precios de los vehículos.

RegistrationYear:
El año de registro varía desde 1000 hasta 9999. Sin embargo, el 75% de los vehículos se registraron entre 1999 y 2008, con una media de 2004.23. Los valores extremos como 1000 y 9999 podrían ser errores o valores atípicos.

Power:
La potencia del motor también muestra una variabilidad significativa, con valores desde 0 hasta 20,000. La media es de 110.09, pero la desviación estándar es alta (189.85), lo que sugiere que hay vehículos con potencias muy altas que podrían ser atípicos o errores en los datos.

Mileage:
El kilometraje varía ampliamente entre los vehículos, con un promedio de 128,211.17 y una desviación estándar de 37,905.34. Esto indica que algunos vehículos tienen un kilometraje muy alto, mientras que otros tienen valores más bajos.

RegistrationMonth:
La mayoría de los vehículos tienen meses de registro entre 3 y 9, con una media de 5.71. La existencia de valores cero podría indicar datos faltantes o errores.

NumberOfPictures:
Esta columna no aporta ninguna información útil, ya que todos los valores son cero.

PostalCode:
Los códigos postales varían entre 1,067 y 99,998, con una media de 50,508.69 y una desviación estándar de 25,783.10. Esto refleja la diversidad geográfica de los vehículos en el conjunto de datos.


Vamos a incorporar el análisis de las variables RegistrationYear y Power. Vamos a comenzar con la columna de RegistrationYear.

RegistrationYear: El año de registro varía desde 1000 hasta 9999. Sin embargo, el 75% de los vehículos se registraron entre 1999 y 2008, con una media de 2004.23. Los valores extremos como 1000 y 9999 podrían ser errores o valores atípicos.

In [9]:
# Obtenemos sus valores únicos y los ordenamos
unique_years = np.sort(vehicles['RegistrationYear'].unique())

# Imprimimos los valores ordenados
print("Valores únicos ordenados:",unique_years)

Valores únicos ordenados: [1000 1001 1039 1111 1200 1234 1253 1255 1300 1400 1500 1600 1602 1688
 1800 1910 1915 1919 1920 1923 1925 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1940 1941 1942 1943 1944 1945 1946 1947 1948
 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962
 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2066 2200 2222 2290 2500 2800 2900 3000 3200 3500 3700 3800 4000
 4100 4500 4800 5000 5300 5555 5600 5900 5911 6000 6500 7000 7100 7500
 7800 8000 8200 8455 8500 8888 9000 9229 9450 9996 9999]


Una vez identificados los valores atípicos de la variable RegistrationYear, vamos a definir un rango de años válidos (por ejemplo entre 1900 y 2024), y los valores que estén fuera de ese rango los vamos a reemplazar por un valor de -1.

In [10]:
# Definimos el rango de años válidos (por ejemplo, 1900 a 2023)
valid_years_range = (1900, 2024)

# Reemplazamos los valores atípicos en RegistrationYear con -1.
vehicles.loc[~vehicles['RegistrationYear'].between(valid_years_range[0], valid_years_range[1]), 'RegistrationYear'] = -1

# Verificamos los valores únicos ordenados después de la limpieza
cleaned_unique_years = np.sort(vehicles['RegistrationYear'].unique())
print("Valores únicos ordenados después de eliminar los valores atípicos:")
print(cleaned_unique_years)

Valores únicos ordenados después de eliminar los valores atípicos:
[  -1 1910 1915 1919 1920 1923 1925 1927 1928 1929 1930 1931 1932 1933
 1934 1935 1936 1937 1938 1940 1941 1942 1943 1944 1945 1946 1947 1948
 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962
 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976
 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019]


Ahora vamos a trabajar con la columna de Power.

Power: La potencia del motor también muestra una variabilidad significativa, con valores desde 0 hasta 20,000. La media es de 110.09, pero la desviación estándar es alta (189.85), lo que sugiere que hay vehículos con potencias muy altas que podrían ser atípicos o errores en los datos.

In [11]:
# Obtenemos sus valores únicos y los ordenamos
unique_power = np.sort(vehicles['Power'].unique())

# Imprimimos los valores ordenados
print("Valores únicos ordenados de Power:",unique_power)

Valores únicos ordenados de Power: [    0     1     2     3     4     5     6     7     8     9    10    11
    12    13    14    15    16    17    18    19    20    21    22    23
    24    25    26    27    28    29    30    31    32    33    34    35
    36    37    38    39    40    41    42    43    44    45    46    47
    48    49    50    51    52    53    54    55    56    57    58    59
    60    61    62    63    64    65    66    67    68    69    70    71
    72    73    74    75    76    77    78    79    80    81    82    83
    84    85    86    87    88    89    90    91    92    93    94    95
    96    97    98    99   100   101   102   103   104   105   106   107
   108   109   110   111   112   113   114   115   116   117   118   119
   120   121   122   123   124   125   126   127   128   129   130   131
   132   133   134   135   136   137   138   139   140   141   142   143
   144   145   146   147   148   149   150   151   152   153   154   155
   156   157   1

El rango normal de la variable "Power" (potencia en CV) de un vehículo puede variar considerablemente según el tipo y el tamaño del vehículo. Sin embargo, un rango general que puede servir como referencia es el siguiente:

- Vehículos pequeños y compactos: Por lo general, tienen potencias que van desde aproximadamente 60 CV hasta 150 CV.
- Sedanes y familiares: Suelen tener potencias que van desde 100 CV hasta 250 CV, dependiendo del tamaño del motor y la configuración.
- Vehículos deportivos y de alto rendimiento: Pueden tener potencias que van desde 200 CV hasta más de 600 CV, dependiendo del modelo y la marca.

Considerando esta información, vamos a considerar como rango válido para la variable Power valores mayores de 10CV y menores de 1000 CV, y los valores que estén fuera de ese rango los vamos a reemplazar por un valor de -1.

In [12]:
# Definimos el rango de Power válido:
valid_power_range = (10, 1000)

# Reemplazamos los valores atípicos en Power con -1.
vehicles.loc[~vehicles['Power'].between(valid_power_range[0], valid_power_range[1]), 'Power'] = -1

# Verificamos los valores únicos ordenados después de la limpieza
cleaned_unique_power = np.sort(vehicles['Power'].unique())
print("Valores únicos ordenados después de eliminar los valores atípicos:")
print(cleaned_unique_power)

Valores únicos ordenados después de eliminar los valores atípicos:
[  -1   10   11   12   13   14   15   16   17   18   19   20   21   22
   23   24   25   26   27   28   29   30   31   32   33   34   35   36
   37   38   39   40   41   42   43   44   45   46   47   48   49   50
   51   52   53   54   55   56   57   58   59   60   61   62   63   64
   65   66   67   68   69   70   71   72   73   74   75   76   77   78
   79   80   81   82   83   84   85   86   87   88   89   90   91   92
   93   94   95   96   97   98   99  100  101  102  103  104  105  106
  107  108  109  110  111  112  113  114  115  116  117  118  119  120
  121  122  123  124  125  126  127  128  129  130  131  132  133  134
  135  136  137  138  139  140  141  142  143  144  145  146  147  148
  149  150  151  152  153  154  155  156  157  158  159  160  161  162
  163  164  165  166  167  168  169  170  171  172  173  174  175  176
  177  178  179  180  181  182  183  184  185  186  187  188  189  190
  191  192

A partir del análisis inicial que hemos realizado de los datos, vamos a eliminar del análisis algunas columnas que no proporcionan una información relavante para predecir el precio (Price). 

Esto ayuda a simplificar el conjunto de datos y potencialmente mejorar la eficiencia del modelado.
Como por ejemplo la columna NumberOfPictures que se encuentra en cero o la columna con la información del código postal PostalCode. 

In [13]:
# Definimos las columnas a eliminar
drop_columns = [
    "DateCrawled",
    "DateCreated",
    "PostalCode",
    "LastSeen",
    "RegistrationMonth",
    "NumberOfPictures",
    "Model"
]

A continuación definimos las características como todas las columnas menos las columnas eliminadas y el objetivo (Price). Esto asegura que solo estamos utilizando las variables relevantes para entrenar nuestros modelos.

In [14]:
# Definimos la variable objetivo y las características:
target = "Price"
features = [c for c in vehicles.columns if c not in drop_columns + [target]]

In [15]:
# Mostramos los tipos de datos de las características seleccionadas
vehicles[features].dtypes

VehicleType         object
RegistrationYear     int64
Gearbox             object
Power                int64
Mileage              int64
FuelType            object
Brand               object
NotRepaired         object
dtype: object

In [16]:
# Imprimimos los valores únicos de la variable 'Model'.
print(vehicles['Model'].unique())

['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore' 's_max' 'a2' 'galaxy' 'c3' 'viano' 's_klasse'
 '1_reihe' 'avensis' 'roomster' 'sl' 'kaefer' 'santa' 'cooper' 'leon'
 '4

In [17]:
# Contamos el número de valores únicos en la columna 'Model':
num_unique_models = vehicles['Model'].nunique()
print(f"Número total de valores únicos en la columna 'Model': {num_unique_models}")

Número total de valores únicos en la columna 'Model': 250


In [18]:
# Identificamos las variables categóricas y numéricas:
categorical_features = []
for col in vehicles[features].columns:
    if vehicles[col].dtype == "object":
        categorical_features.append(col)
        
numerical_features = [f for f in features if f not in categorical_features]

In [19]:
# Mostramos las variables categóricas y numéricas
print("Variables categóricas:", categorical_features)
print("Variables numéricas:", numerical_features)

Variables categóricas: ['VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired']
Variables numéricas: ['RegistrationYear', 'Power', 'Mileage']


## ## Subconjunto sin Variable Model y OHE encoding

Realizamos la generación de los Conjuntos de Entrenamiento y Prueba.

### Generación de los Conjuntos de Entrenamiento y Prueba.

In [20]:
# Definimos las columnas a eliminar
drop_columns = [
    "DateCrawled",
    "DateCreated",
    "PostalCode",
    "LastSeen",
    "RegistrationMonth",
    "NumberOfPictures"
]

# Eliminamos las columnas irrelevantes del DataFrame original
vehicles_without_model = vehicles.drop(columns=drop_columns + ["Model"])

In [21]:
# Generación de los Conjuntos de Entrenamiento y Prueba.
train_df, test_df = train_test_split(
    vehicles_without_model,
    test_size = 0.2,
    random_state = 88
)

Este código divide el conjunto de datos vehicles en conjuntos de entrenamiento (train_df) y prueba (test_df). Verificamos que las formas sean correctas para asegurarnos de que se han dividido correctamente.

In [22]:
# Imprimimos las formas (shapes) de los conjuntos de datos resultantes, para verificar sus tamaños.
print("Forma del Dataframe:", vehicles_without_model.shape)
print("Forma del conjunto de entrenamiento:", train_df.shape)
print("Forma del conjunto de prueba:", test_df.shape)

Forma del Dataframe: (354369, 9)
Forma del conjunto de entrenamiento: (283495, 9)
Forma del conjunto de prueba: (70874, 9)


In [23]:
train_df.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Mileage,FuelType,Brand,NotRepaired
85043,4000,bus,2009,manual,80,150000,gasoline,skoda,no
44822,0,sedan,2000,auto,193,150000,petrol,bmw,no
83964,4000,small,2004,manual,60,40000,petrol,fiat,no
191736,1499,NaN,2018,auto,-1,150000,gasoline,audi,NaN
210970,1800,wagon,2001,manual,122,150000,gasoline,mercedes_benz,no


### Imputación de Valores Faltantes.

A continuación realizamos la imputación de Valores Faltantes.

In [24]:
# Creamos un imputador usando la mediana para valores numéricos
imputer = SimpleImputer(strategy='median')

# Ajustamos el imputador con los datos de entrenamiento
imputer.fit(train_df[numerical_features])

# Aplicamos la imputación a los conjuntos de entrenamiento y prueba
train_df_nonulls = imputer.transform(train_df[numerical_features])
test_df_nonulls = imputer.transform(test_df[numerical_features])

# Convertimos los resultados en DataFrames
train_df_nonulls = pd.DataFrame(train_df_nonulls, columns=numerical_features)
test_df_nonulls = pd.DataFrame(test_df_nonulls, columns=numerical_features)


In [25]:
train_df_nonulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283495 entries, 0 to 283494
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   RegistrationYear  283495 non-null  float64
 1   Power             283495 non-null  float64
 2   Mileage           283495 non-null  float64
dtypes: float64(3)
memory usage: 6.5 MB


In [26]:
test_df_nonulls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70874 entries, 0 to 70873
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RegistrationYear  70874 non-null  float64
 1   Power             70874 non-null  float64
 2   Mileage           70874 non-null  float64
dtypes: float64(3)
memory usage: 1.6 MB


### Codificación One-Hot de Variables Categóricas.

A continuación realizamos la codificación One-Hot de Variables Categóricas.

In [27]:
# Creamos un codificador One-Hot con drop='first' para evitar la multicolinealidad
encoder = OneHotEncoder(drop='first', sparse=False)

# Ajustamos el codificador con los datos de entrenamiento categóricos
encoder.fit(train_df[categorical_features])

# Aplicamos la codificación One-Hot a los conjuntos de entrenamiento y prueba
train_df_ohe = encoder.transform(train_df[categorical_features])
test_df_ohe = encoder.transform(test_df[categorical_features])

# Convertimos los resultados en DataFrames y añadimos las columnas numéricas imputadas
train_df_ohe = pd.DataFrame(train_df_ohe, columns=encoder.get_feature_names(categorical_features))
test_df_ohe = pd.DataFrame(test_df_ohe, columns=encoder.get_feature_names(categorical_features))


In [28]:
# Añadimos las variables numéricas imputadas a los DataFrames codificados
for f in numerical_features:
    train_df_ohe[f] = train_df_nonulls[f]
    test_df_ohe[f] = test_df_nonulls[f]

Hemos aplicado la codificación One-Hot a las variables categóricas (categorical_features) y luego añadimos las variables numéricas imputadas (train_df_nonulls y test_df_nonulls) a los DataFrames codificados.

In [29]:
train_df_ohe.head()

,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,VehicleType_nan,Gearbox_manual,Gearbox_nan,...,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes,NotRepaired_nan,RegistrationYear,Power,Mileage
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2009.0,80.0,150000.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,193.0,150000.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2004.0,60.0,40000.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2018.0,-1.0,150000.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2001.0,122.0,150000.0


In [30]:
test_df_ohe.head()

,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_wagon,VehicleType_nan,Gearbox_manual,Gearbox_nan,...,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes,NotRepaired_nan,RegistrationYear,Power,Mileage
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1997.0,-1.0,150000.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1995.0,150.0,150000.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1995.0,101.0,150000.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1998.0,75.0,150000.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2006.0,105.0,150000.0


In [31]:
# Realizamos la verificación de Valores Faltantes
train_df_ohe.isna().sum().sum()

0

Mediante este código verificamos que no haya valores faltantes en el DataFrame train_df_ohe después de la imputación y codificación.

In [32]:
# Imprimimos las formas (shapes) de los conjuntos de datos resultantes, para verificar sus tamaños.
print("Forma del Dataframe:", vehicles_without_model.shape)
print("Forma del conjunto de entrenamiento:", train_df_ohe.shape)
print("Forma del conjunto de prueba:", test_df_ohe.shape)

Forma del Dataframe: (354369, 9)
Forma del conjunto de entrenamiento: (283495, 61)
Forma del conjunto de prueba: (70874, 61)


Definimos las variables predictoras (X_train, X_test) y la variable objetivo (y_train, y_test) 

In [33]:
# Variables predictoras
X_train = train_df_ohe
X_test = test_df_ohe

# Variable objetivo
y_train = train_df['Price']
y_test = test_df['Price']

In [34]:
# Imprimimos las formas (shapes) de los conjuntos de datos resultantes, para verificar sus tamaños.
print("Forma del Dataframe:", vehicles_without_model.shape)
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de prueba:", X_test.shape)

Forma del Dataframe: (354369, 9)
Forma del conjunto de entrenamiento: (283495, 61)
Forma del conjunto de prueba: (70874, 61)


## Subconjunto con Variable Model y Ordinal Encoder

In [35]:
# Creamos un nuevo conjunto de datos incluyendo la variable 'Model'

# Definimos las columnas a eliminar (excluyendo 'Model')
drop_columns_with_model = [
    "DateCrawled",
    "DateCreated",
    "PostalCode",
    "LastSeen",
    "RegistrationMonth",
    "NumberOfPictures"
]

# Creamos una copia del DataFrame y eliminamos las columnas irrelevantes
vehicles_with_model = vehicles.drop(columns=drop_columns_with_model)

# Identificar las variables categóricas y numéricas

# Definimos la variable objetivo y las características
target = "Price"
features_with_model = [c for c in vehicles_with_model.columns if c != target]

# Mostramos los tipos de datos de las características seleccionadas
vehicles[features_with_model].dtypes

VehicleType         object
RegistrationYear     int64
Gearbox             object
Power                int64
Model               object
Mileage              int64
FuelType            object
Brand               object
NotRepaired         object
dtype: object

In [36]:
# Identificamos las variables categóricas y numéricas
categorical_features_with_model = []
for col in vehicles_with_model[features_with_model].columns:
    if vehicles_with_model[col].dtype == "object":
        categorical_features_with_model.append(col)

numerical_features_with_model = [f for f in features_with_model if f not in categorical_features_with_model]

# Mostramos las variables categóricas y numéricas
print("Variables categóricas con 'Model':", categorical_features_with_model)
print("Variables numéricas con 'Model':", numerical_features_with_model)

Variables categóricas con 'Model': ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
Variables numéricas con 'Model': ['RegistrationYear', 'Power', 'Mileage']


In [37]:
# Generación de los Conjuntos de Entrenamiento y Prueba
train_df_with_model, test_df_with_model = train_test_split(
    vehicles_with_model,
    test_size=0.2,
    random_state=88
)

# Imprimimos las formas (shapes) de los conjuntos de datos resultantes, para verificar sus tamaños
print("Forma del Dataframe:", vehicles_with_model.shape)
print("Forma del conjunto de entrenamiento:", train_df_with_model.shape)
print("Forma del conjunto de prueba:", test_df_with_model.shape)

Forma del Dataframe: (354369, 10)
Forma del conjunto de entrenamiento: (283495, 10)
Forma del conjunto de prueba: (70874, 10)


In [38]:
vehicles_with_model.head()

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,-1,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no


In [39]:
from sklearn.impute import SimpleImputer

# Definimos el imputador para las variables numéricas, reemplazando NaN con la mediana
numerical_imputer = SimpleImputer(strategy='median')

# Ajustamos el imputador con los datos de entrenamiento numéricos
numerical_imputer.fit(train_df_with_model[numerical_features_with_model])

# Aplicamos la imputación a los conjuntos de entrenamiento y prueba
train_df_numerical_nonulls = numerical_imputer.transform(train_df_with_model[numerical_features_with_model])
test_df_numerical_nonulls = numerical_imputer.transform(test_df_with_model[numerical_features_with_model])

# Convertimos los resultados en DataFrames
train_df_numerical_nonulls = pd.DataFrame(train_df_numerical_nonulls, columns=numerical_features_with_model)
test_df_numerical_nonulls = pd.DataFrame(test_df_numerical_nonulls, columns=numerical_features_with_model)

# Definimos el imputador para las variables categóricas, reemplazando NaN con 'missing'
categorical_imputer = SimpleImputer(strategy='constant', fill_value='missing')

# Ajustamos el imputador con los datos de entrenamiento categóricos
categorical_imputer.fit(train_df_with_model[categorical_features_with_model])

# Aplicamos la imputación a los conjuntos de entrenamiento y prueba
train_df_categorical_nonulls = categorical_imputer.transform(train_df_with_model[categorical_features_with_model])
test_df_categorical_nonulls = categorical_imputer.transform(test_df_with_model[categorical_features_with_model])

# Convertimos los resultados en DataFrames
train_df_categorical_nonulls = pd.DataFrame(train_df_categorical_nonulls, columns=categorical_features_with_model)
test_df_categorical_nonulls = pd.DataFrame(test_df_categorical_nonulls, columns=categorical_features_with_model)

# Definimos y ajustamos el OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

# Ajustamos el codificador con los datos de entrenamiento categóricos
ordinal_encoder.fit(train_df_categorical_nonulls)

# Aplicamos la codificación Ordinal a los conjuntos de entrenamiento y prueba
train_df_with_model_encoded = ordinal_encoder.transform(train_df_categorical_nonulls)
test_df_with_model_encoded = ordinal_encoder.transform(test_df_categorical_nonulls)

# Convertimos los resultados en DataFrames y añadimos las columnas numéricas imputadas
train_df_with_model_encoded = pd.DataFrame(train_df_with_model_encoded, columns=categorical_features_with_model)
test_df_with_model_encoded = pd.DataFrame(test_df_with_model_encoded, columns=categorical_features_with_model)

# Añadimos las variables numéricas imputadas a los DataFrames codificados
for f in numerical_features_with_model:
    train_df_with_model_encoded[f] = train_df_numerical_nonulls[f].values
    test_df_with_model_encoded[f] = test_df_numerical_nonulls[f].values

# Imprimimos las formas (shapes) de los conjuntos de datos resultantes, para verificar sus tamaños
print("Forma del conjunto de entrenamiento con 'Model':", train_df_with_model_encoded.shape)
print("Forma del conjunto de prueba con 'Model':", test_df_with_model_encoded.shape)


Forma del conjunto de entrenamiento con 'Model': (283495, 9)
Forma del conjunto de prueba con 'Model': (70874, 9)


In [40]:
# Realizamos la verificación de Valores Faltantes
train_df_with_model_encoded.isna().sum().sum()

0

In [42]:
# Definir las variables predictoras y la variable objetivo para el conjunto con 'Model'
features_with_model = [c for c in train_df_with_model_encoded.columns if c != target]

X_train_with_model = train_df_with_model_encoded[features_with_model]
X_test_with_model = test_df_with_model_encoded[features_with_model]

y_train_with_model = train_df_with_model[target]
y_test_with_model = test_df_with_model[target]

# Verificar las formas (shapes) de los conjuntos de datos resultantes
print("Forma de X_train con 'Model':", X_train_with_model.shape)
print("Forma de X_test con 'Model':", X_test_with_model.shape)
print("Forma de y_train con 'Model':", y_train_with_model.shape)
print("Forma de y_test con 'Model':", y_test_with_model.shape)

Forma de X_train con 'Model': (283495, 9)
Forma de X_test con 'Model': (70874, 9)
Forma de y_train con 'Model': (283495,)
Forma de y_test con 'Model': (70874,)


## Modelo Regresión Lineal

En el caso del modelo de regresión lineal, se recomienda solo usar el OHE encoding.

In [45]:
%%time
# Inicializamos el modelo
model_lr = LinearRegression()

# Entrenamos el modelo
model_lr.fit(X_train, y_train)

# Predecimos los precios en el conjunto de prueba
y_pred_lr = model_lr.predict(X_test)

# Evaluamos el modelo:
print(f"""
Linear Regression performance:
{round(mean_squared_error(y_test, y_pred_lr)**0.5, 4)}
""")


Linear Regression performance:
3138.5415

CPU times: user 1.77 s, sys: 903 ms, total: 2.67 s
Wall time: 2.65 s


El valor de la raíz del error cuadrático medio (RMSE) obtenido para el modelo de regresión lineal es de 3138.5415. Este valor indica la magnitud promedio del error entre los valores predichos y los valores reales de precios de los vehículos.

Un RMSE de 3138.5415 sugiere que, en promedio, las predicciones del modelo de regresión lineal están desviadas del valor real del precio del vehículo en aproximadamente 3138.54 unidades monetarias. Esto puede considerarse un error relativamente alto, dado que el precio promedio de los vehículos en el conjunto de datos es de aproximadamente 4416.66 (según el análisis descriptivo inicial).

Es base a estos resultados, vemos que se hace necesario el análisis de los otros modelos que seguramente podrán entregarnos un mejor desempeño. El modelo de Regresión Lineal nos permite establecer una línea base de desempeño con la cual se pueden comparar otros modelos más complejos. Si un modelo más avanzado no supera a la regresión lineal, esto puede indicar problemas en el modelo avanzado o en los datos.

## Modelo Bosque Aleatorio (conjunto sin variable Model).

A continuación vamos a ocupar el modelo de Bosque Aleatorio RandomForestRegressor, on el subconjunto de datos sin la variable Model.

Vamos a explorar diferentes combinaciones de hiperparámetros y encontrar la que maximice el rendimiento del modelo.

Para ello vamos a utilizar un bucle for anidado para iterar sobre diferentes valores de dos hiperparámetros del modelo de Bosque Aleatorio: el número de estimadores (n_estimators) y la profundidad máxima del árbol (max_depth). 
Ocuparemos valores de n_estimators (10, 50, 100) y max_depth (5, 10, 15).

In [53]:
%%time
# Inicializar el mejor puntaje y los mejores hiperparámetros
max_score = float('inf')
best_params = {'n_estimators': None, 'max_depth': None}

# Bucle para ajustar los hiperparámetros
for n_estimators in [10, 50, 100]:
    for max_depth in [5, 10, 15]:
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=88)
        model.fit(X_train, y_train)
        y_pred_rf = model.predict(X_test)
        score = mean_squared_error(y_test, y_pred_rf)**0.5

        if score < max_score:
            max_score = score
            best_params['n_estimators'] = n_estimators
            best_params['max_depth'] = max_depth
            print(f"""
            N estimators: {n_estimators}
            Max depth: {max_depth}
            RMSE: {round(score, 4)}
            """)

# Entrenar el modelo con los mejores hiperparámetros encontrados
model_rf_best = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], random_state=88)
model_rf_best.fit(X_train, y_train)
y_pred_rf_best = model_rf_best.predict(X_test)

# Evaluar el modelo
print(f"""
Optimized Random Forest performance:
{round(mean_squared_error(y_test, y_pred_rf_best)**0.5, 4)}
""")


            N estimators: 10
            Max depth: 5
            RMSE: 2495.3984
            

            N estimators: 10
            Max depth: 10
            RMSE: 2048.3717
            

            N estimators: 10
            Max depth: 15
            RMSE: 1852.5917
            

            N estimators: 50
            Max depth: 15
            RMSE: 1830.1057
            

            N estimators: 100
            Max depth: 15
            RMSE: 1827.218
            

Optimized Random Forest performance:
1827.218

CPU times: user 13min 6s, sys: 567 ms, total: 13min 7s
Wall time: 13min 7s


La mejor configuración encontrada para el modelo de Bosque Aleatorio fue con n_estimators = 100 y max_depth = 15, obteniendo un RMSE de 1827.218. Esta configuración se logró después de evaluar varias combinaciones, y resultó ser la más efectiva en términos de precisión.

Se observa una mejora en el RMSE al aumentar tanto el número de estimadores como la profundidad máxima del modelo. Esto indica que el modelo se beneficia de una mayor capacidad para capturar la variabilidad en los datos.

La configuración óptima con n_estimators = 100 y max_depth = 15 ofrece el mejor equilibrio entre precisión y complejidad del modelo. Aunque incrementos adicionales en los hiperparámetros podrían seguir mejorando el rendimiento, también aumentarían significativamente el tiempo de entrenamiento y el riesgo de sobreajuste.

Con respecto al tiempo total de cómputo para la búsqueda de hiperparámetros fue:
- CPU times: 13min 6s
- Wall time: 13min 7s

Estos tiempos reflejan la complejidad computacional asociada con la búsqueda de hiperparámetros en modelos de Bosque Aleatorio, especialmente con configuraciones más grandes.

El modelo de Bosque Aleatorio con los hiperparámetros optimizados logró un RMSE de 1827.218, lo que representa una mejora significativa en comparación con el modelo de Regresión lineal. Sin embargo, el costo computacional es considerable, y es importante equilibrar la precisión del modelo con los recursos disponibles.

## Modelo Bosque Aleatorio (conjunto con variable Model).

A continuación vamos a ocupar el modelo de Bosque Aleatorio RandomForestRegressor pero con el subconjunto de datos con la variable Model.

In [55]:
%%time
# Inicializar el mejor puntaje y los mejores hiperparámetros
max_score_model = float('inf')
best_params_model = {'n_estimators': None, 'max_depth': None}

# Bucle para ajustar los hiperparámetros
for n_estimators in [10, 50, 100]:
    for max_depth in [5, 10, 15]:
        model_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=88)
        model_model.fit(X_train_with_model, y_train_with_model)
        y_pred_rf_model = model_model.predict(X_test_with_model)
        score_model = mean_squared_error(y_test_with_model, y_pred_rf_model)**0.5

        if score_model < max_score_model:
            max_score_model = score_model
            best_params_model['n_estimators'] = n_estimators
            best_params_model['max_depth'] = max_depth
            print(f"""
            N estimators: {n_estimators}
            Max depth: {max_depth}
            RMSE: {round(score_model, 4)}
            """)

# Entrenar el modelo con los mejores hiperparámetros encontrados
model_rf_best_model = RandomForestRegressor(n_estimators=best_params_model['n_estimators'], max_depth=best_params_model['max_depth'], random_state=88)
model_rf_best_model.fit(X_train_with_model, y_train_with_model)
y_pred_rf_best_model = model_rf_best_model.predict(X_test_with_model)

# Evaluar el modelo
print(f"""
Optimized Random Forest performance with 'Model':
{round(mean_squared_error(y_test_with_model, y_pred_rf_best_model)**0.5, 4)}
""")


            N estimators: 10
            Max depth: 5
            RMSE: 2498.2654
            

            N estimators: 10
            Max depth: 10
            RMSE: 2034.4598
            

            N estimators: 10
            Max depth: 15
            RMSE: 1812.0848
            

            N estimators: 50
            Max depth: 15
            RMSE: 1787.3712
            

            N estimators: 100
            Max depth: 15
            RMSE: 1782.74
            

Optimized Random Forest performance with 'Model':
1782.74

CPU times: user 6min 38s, sys: 390 ms, total: 6min 38s
Wall time: 6min 39s


La inclusión de la variable 'Model' ha mejorado significativamente el rendimiento del modelo, reduciendo el RMSE a 1782.74 en el mejor caso, en comparación con 1827.218 sin la variable 'Model'. 

Esto demuestra que 'Model' es una característica relevante y útil para la predicción del precio.

Con respecto al tiempo de ejecución, la inclusión de la variable 'Model' no solo mejora el rendimiento del modelo en términos de RMSE, sino que también disminuye casi en un 50% el tiempo de ejecución. 

Esto indica que 'Model' proporciona información adicional relevante que facilita la predicción y permite al modelo converger más rápidamente.

## Modelo LightGBM (conjunto sin variable Model).

A continuación vamos a ocupar el modelo de LightGBM en el subconjunto de datos sin la variable Model.

Vamos a comenzar nuestro análisis con un modelo simple de LghtGBM con los parámetros por defecto. 

In [54]:
%%time

# Inicializamos el modelo LightGBM con parámetros por defecto
model_lgb_simple = lgb.LGBMRegressor(random_state=88)

# Entrenamos el modelo
model_lgb_simple.fit(X_train, y_train)

# Predecimos los precios en el conjunto de prueba
y_pred_lgb_simple = model_lgb_simple.predict(X_test)

# Evaluamos el modelo
print(f"""
Simple LightGBM performance:
{round(mean_squared_error(y_test, y_pred_lgb_simple)**0.5, 4)}
""")


Simple LightGBM performance:
1867.9611

CPU times: user 5.15 s, sys: 60.1 ms, total: 5.21 s
Wall time: 5.23 s


El RMSE obtenido para el modelo simple fue de 1867.9611. Este resultado ya es bastante prometedor y sugiere que LightGBM puede capturar patrones importantes en los datos.

La eficiencia computacional del modelo es notable, ya que solo tomó alrededor de 5 segundos en tiempo de CPU y Wall para entrenar y hacer predicciones.

Para mejorar el rendimiento del modelo LightGBM, vamos a realizar una búsqueda de hiperparámetros utilizando GridSearchCV.

In [39]:
%%time

# Definimos el modelo LightGBM
model_lgb = lgb.LGBMRegressor(random_state=88)

# Definimos un conjunto de hiperparámetros para la búsqueda
param_grid = {
    'num_leaves': [31, 40],
    'max_depth': [10, 20],
    'learning_rate': [0.1, 0.05],
    'n_estimators': [100, 200]
}

# Realizamos la búsqueda con GridSearchCV
grid_search = GridSearchCV(
    estimator=model_lgb,
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Entrenamos el GridSearchCV
grid_search.fit(X_train, y_train)

# Obtenemos el mejor modelo
best_model = grid_search.best_estimator_

# Imprimimos los mejores hiperparámetros
print(f"Best parameters found: {grid_search.best_params_}")


Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters found: {'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 200, 'num_leaves': 40}
CPU times: user 5min 4s, sys: 3.87 s, total: 5min 8s
Wall time: 5min 10s


In [40]:
%%time
# Predecimos los precios en el conjunto de prueba con el mejor modelo
y_pred_lgb_best = best_model.predict(X_test)

# Evaluamos el modelo
print(f"""
Optimized LightGBM performance:
{round(mean_squared_error(y_test, y_pred_lgb_best)**0.5, 4)}
""")

# Imprimimos los mejores hiperparámetros
print(f"Best parameters found: {grid_search.best_params_}")


Optimized LightGBM performance:
1801.0548

Best parameters found: {'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 200, 'num_leaves': 40}
CPU times: user 1.46 s, sys: 19.9 ms, total: 1.48 s
Wall time: 1.41 s


El RMSE mejoró a 1801.0548 con el modelo optimizado, una reducción significativa respecto al modelo simple. Esto indica que la búsqueda de hiperparámetros fue efectiva y permitió mejorar la precisión del modelo.

Los mejores parámetros encontrados fueron {'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 200, 'num_leaves': 40}. Estos parámetros sugieren que un mayor número de árboles, mayor profundidad y un número de hojas más grande son beneficiosos para este conjunto de datos.

La búsqueda de los mejores hiperparámetros mediante GridSearchCV tomó aproximadamente 5 minutos y 10 segundos de tiempo de pared. Esto es razonable dado el número de combinaciones probadas (48 en total) y la complejidad del proceso de validación cruzada.

Por otro lado, la predicción utilizando el modelo optimizado tomó apenas 1.4 segundos de tiempo de pared. Esto muestra la eficiencia del modelo LightGBM una vez que está entrenado, permitiendo predicciones rápidas incluso en conjuntos de datos relativamente grandes.

## Modelo LightGBM (conjunto con variable Model).

A continuación vamos a ocupar el modelo de LightGBM en el subconjunto de datos con la variable Model.

Ejecución de un Modelo Simple de LightGBM

In [57]:
%%time
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Inicializamos el modelo LightGBM con parámetros por defecto
model_lgb_simple = lgb.LGBMRegressor(random_state=88)

# Entrenamos el modelo
model_lgb_simple.fit(X_train_with_model, y_train_with_model)

# Predecimos los precios en el conjunto de prueba
y_pred_lgb_simple_with_model = model_lgb_simple.predict(X_test_with_model)

# Evaluamos el modelo
print(f"""
Simple LightGBM performance with 'Model':
{round(mean_squared_error(y_test_with_model, y_pred_lgb_simple_with_model)**0.5, 4)}
""")



Simple LightGBM performance with 'Model':
1848.976

CPU times: user 4.74 s, sys: 12.1 ms, total: 4.76 s
Wall time: 4.7 s


In [58]:
%%time

# Definimos el modelo LightGBM
model_lgb_with_model = lgb.LGBMRegressor(random_state=88)

# Definimos un conjunto de hiperparámetros para la búsqueda
param_grid_with_model = {
    'num_leaves': [31, 40],
    'max_depth': [10, 20],
    'learning_rate': [0.1, 0.05],
    'n_estimators': [100, 200]
}

# Realizamos la búsqueda con GridSearchCV
grid_search_with_model = GridSearchCV(
    estimator=model_lgb_with_model,
    param_grid=param_grid_with_model,
    cv=3,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Entrenamos el GridSearchCV
grid_search_with_model.fit(X_train_with_model, y_train_with_model)

# Obtenemos el mejor modelo
best_model_with_model = grid_search_with_model.best_estimator_

# Imprimimos los mejores hiperparámetros
print(f"Best parameters found: {grid_search_with_model.best_params_}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best parameters found: {'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 200, 'num_leaves': 40}
CPU times: user 4min 46s, sys: 2.15 s, total: 4min 48s
Wall time: 4min 49s


In [59]:
%%time

# Predecimos los precios en el conjunto de prueba con el mejor modelo
y_pred_lgb_best_with_model = best_model_with_model.predict(X_test_with_model)

# Evaluamos el modelo
print(f"""
Optimized LightGBM performance with 'Model':
{round(mean_squared_error(y_test_with_model, y_pred_lgb_best_with_model)**0.5, 4)}
""")

# Imprimimos los mejores hiperparámetros
print(f"Best parameters found: {grid_search_with_model.best_params_}")



Optimized LightGBM performance with 'Model':
1770.7392

Best parameters found: {'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 200, 'num_leaves': 40}
CPU times: user 1.32 s, sys: 16 ms, total: 1.33 s
Wall time: 1.31 s


La inclusión de la variable 'Model' mejoró el rendimiento del modelo LightGBM, reduciendo el RMSE de 1801.0548 a 1770.7392. Esta mejora en el RMSE indica que el modelo con la variable 'Model' tiene una mayor precisión en la predicción de los precios de los vehículos.

La búsqueda de hiperparámetros fue ligeramente más rápida cuando se incluyó la variable 'Model', con una reducción en el tiempo total de 5min 8s a 4min 48s.

El tiempo de predicción con el mejor modelo fue prácticamente el mismo para ambos conjuntos de datos, con una ligera ventaja para el conjunto con la variable 'Model' (1.31s frente a 1.41s).

## Modelo CatBoost  (conjunto sin variable model).

A continuación vamos a ocupar el modelo de CatBoostRegressor.

En el caso del CatBoostRegressor, no es necesario aplicar el One Hot Encoder a las características categóricas por separado antes de entrenar el modelo. CatBoost tiene la capacidad incorporada de manejar características categóricas de manera directa.


In [50]:
# Definimos el modelo CatBoostRegressor
model_cb = cb.CatBoostRegressor(loss_function='RMSE', iterations=1000)

In [51]:
%%time
# Entrenamos el modelo
model_cb.fit(
    train_df[features].fillna(-100),
    train_df[target],
    cat_features=categorical_features,
    early_stopping_rounds=5
)

Learning rate set to 0.099927
0:	learn: 4229.3980391	total: 683ms	remaining: 11m 22s
1:	learn: 3984.5435485	total: 1.29s	remaining: 10m 45s
2:	learn: 3766.6319211	total: 1.87s	remaining: 10m 21s
3:	learn: 3572.1927607	total: 2.54s	remaining: 10m 32s
4:	learn: 3408.5353551	total: 3.15s	remaining: 10m 26s
5:	learn: 3262.5011965	total: 3.67s	remaining: 10m 7s
6:	learn: 3132.2889245	total: 4.25s	remaining: 10m 3s
7:	learn: 3019.7591809	total: 4.87s	remaining: 10m 4s
8:	learn: 2916.6755965	total: 5.41s	remaining: 9m 55s
9:	learn: 2824.6334180	total: 5.95s	remaining: 9m 49s
10:	learn: 2738.3400139	total: 6.4s	remaining: 9m 35s
11:	learn: 2666.6504980	total: 6.88s	remaining: 9m 26s
12:	learn: 2601.2061434	total: 7.29s	remaining: 9m 13s
13:	learn: 2546.8287438	total: 7.73s	remaining: 9m 4s
14:	learn: 2498.3076768	total: 8.34s	remaining: 9m 7s
15:	learn: 2459.5240042	total: 8.76s	remaining: 8m 58s
16:	learn: 2418.8062572	total: 9.29s	remaining: 8m 57s
17:	learn: 2380.6270123	total: 9.87s	remain

149:	learn: 1898.2883772	total: 1m 14s	remaining: 6m 59s
150:	learn: 1896.9572125	total: 1m 14s	remaining: 6m 59s
151:	learn: 1896.3093443	total: 1m 15s	remaining: 6m 58s
152:	learn: 1895.3419448	total: 1m 15s	remaining: 6m 58s
153:	learn: 1894.4902185	total: 1m 16s	remaining: 6m 58s
154:	learn: 1893.9492323	total: 1m 16s	remaining: 6m 58s
155:	learn: 1893.1508142	total: 1m 17s	remaining: 6m 58s
156:	learn: 1892.7423556	total: 1m 17s	remaining: 6m 57s
157:	learn: 1892.1793601	total: 1m 18s	remaining: 6m 57s
158:	learn: 1891.6727965	total: 1m 18s	remaining: 6m 56s
159:	learn: 1891.3218997	total: 1m 19s	remaining: 6m 55s
160:	learn: 1890.5254722	total: 1m 19s	remaining: 6m 55s
161:	learn: 1890.0554047	total: 1m 20s	remaining: 6m 55s
162:	learn: 1889.4758693	total: 1m 20s	remaining: 6m 54s
163:	learn: 1888.8719522	total: 1m 21s	remaining: 6m 53s
164:	learn: 1888.3451310	total: 1m 21s	remaining: 6m 53s
165:	learn: 1887.6124290	total: 1m 22s	remaining: 6m 52s
166:	learn: 1886.7955541	total:

294:	learn: 1834.9783322	total: 2m 25s	remaining: 5m 48s
295:	learn: 1834.7257360	total: 2m 26s	remaining: 5m 48s
296:	learn: 1834.4781379	total: 2m 26s	remaining: 5m 47s
297:	learn: 1834.2289850	total: 2m 27s	remaining: 5m 46s
298:	learn: 1834.0161661	total: 2m 27s	remaining: 5m 46s
299:	learn: 1833.7929198	total: 2m 28s	remaining: 5m 45s
300:	learn: 1833.7343509	total: 2m 28s	remaining: 5m 45s
301:	learn: 1833.6347954	total: 2m 29s	remaining: 5m 44s
302:	learn: 1833.3669189	total: 2m 29s	remaining: 5m 44s
303:	learn: 1832.8671223	total: 2m 30s	remaining: 5m 43s
304:	learn: 1832.6037927	total: 2m 30s	remaining: 5m 43s
305:	learn: 1832.3498708	total: 2m 31s	remaining: 5m 42s
306:	learn: 1831.9434713	total: 2m 31s	remaining: 5m 42s
307:	learn: 1831.6897014	total: 2m 32s	remaining: 5m 41s
308:	learn: 1831.5247773	total: 2m 32s	remaining: 5m 41s
309:	learn: 1831.3133895	total: 2m 33s	remaining: 5m 40s
310:	learn: 1831.0738124	total: 2m 33s	remaining: 5m 40s
311:	learn: 1830.9834047	total:

439:	learn: 1800.4525648	total: 3m 37s	remaining: 4m 36s
440:	learn: 1800.2313288	total: 3m 37s	remaining: 4m 36s
441:	learn: 1800.0237170	total: 3m 38s	remaining: 4m 35s
442:	learn: 1799.6566382	total: 3m 38s	remaining: 4m 34s
443:	learn: 1799.4577796	total: 3m 39s	remaining: 4m 34s
444:	learn: 1799.1577396	total: 3m 39s	remaining: 4m 34s
445:	learn: 1798.9224086	total: 3m 40s	remaining: 4m 33s
446:	learn: 1798.6857166	total: 3m 40s	remaining: 4m 32s
447:	learn: 1798.4801101	total: 3m 41s	remaining: 4m 32s
448:	learn: 1798.2456537	total: 3m 41s	remaining: 4m 31s
449:	learn: 1797.9587999	total: 3m 41s	remaining: 4m 31s
450:	learn: 1797.9006318	total: 3m 42s	remaining: 4m 30s
451:	learn: 1797.5852518	total: 3m 42s	remaining: 4m 30s
452:	learn: 1797.3174839	total: 3m 43s	remaining: 4m 29s
453:	learn: 1797.2215706	total: 3m 43s	remaining: 4m 29s
454:	learn: 1796.9458588	total: 3m 44s	remaining: 4m 28s
455:	learn: 1796.7263567	total: 3m 44s	remaining: 4m 28s
456:	learn: 1796.5238469	total:

584:	learn: 1777.1777602	total: 4m 48s	remaining: 3m 24s
585:	learn: 1777.0877349	total: 4m 49s	remaining: 3m 24s
586:	learn: 1776.9760750	total: 4m 49s	remaining: 3m 23s
587:	learn: 1776.7265151	total: 4m 50s	remaining: 3m 23s
588:	learn: 1776.6675014	total: 4m 50s	remaining: 3m 22s
589:	learn: 1776.4174941	total: 4m 51s	remaining: 3m 22s
590:	learn: 1776.1966977	total: 4m 51s	remaining: 3m 21s
591:	learn: 1775.9212099	total: 4m 52s	remaining: 3m 21s
592:	learn: 1775.8841464	total: 4m 52s	remaining: 3m 20s
593:	learn: 1775.6926925	total: 4m 53s	remaining: 3m 20s
594:	learn: 1775.6709771	total: 4m 53s	remaining: 3m 19s
595:	learn: 1775.4286872	total: 4m 54s	remaining: 3m 19s
596:	learn: 1775.2130540	total: 4m 54s	remaining: 3m 19s
597:	learn: 1775.1559026	total: 4m 55s	remaining: 3m 18s
598:	learn: 1774.9843905	total: 4m 55s	remaining: 3m 18s
599:	learn: 1774.7297903	total: 4m 56s	remaining: 3m 17s
600:	learn: 1774.6322316	total: 4m 56s	remaining: 3m 17s
601:	learn: 1774.5864252	total:

729:	learn: 1759.9041131	total: 6m	remaining: 2m 13s
730:	learn: 1759.7905346	total: 6m 1s	remaining: 2m 12s
731:	learn: 1759.6398980	total: 6m 1s	remaining: 2m 12s
732:	learn: 1759.5735549	total: 6m 2s	remaining: 2m 12s
733:	learn: 1759.3897382	total: 6m 2s	remaining: 2m 11s
734:	learn: 1759.2926681	total: 6m 3s	remaining: 2m 11s
735:	learn: 1758.9669384	total: 6m 3s	remaining: 2m 10s
736:	learn: 1758.8078953	total: 6m 4s	remaining: 2m 10s
737:	learn: 1758.7481862	total: 6m 4s	remaining: 2m 9s
738:	learn: 1758.6432569	total: 6m 5s	remaining: 2m 9s
739:	learn: 1758.5651378	total: 6m 5s	remaining: 2m 8s
740:	learn: 1758.5144429	total: 6m 6s	remaining: 2m 8s
741:	learn: 1758.4383811	total: 6m 6s	remaining: 2m 7s
742:	learn: 1758.1783149	total: 6m 7s	remaining: 2m 7s
743:	learn: 1758.0454020	total: 6m 7s	remaining: 2m 6s
744:	learn: 1757.8880916	total: 6m 8s	remaining: 2m 6s
745:	learn: 1757.8853713	total: 6m 8s	remaining: 2m 5s
746:	learn: 1757.8808349	total: 6m 9s	remaining: 2m 5s
747:	

875:	learn: 1745.6427523	total: 7m 13s	remaining: 1m 1s
876:	learn: 1745.5845291	total: 7m 13s	remaining: 1m
877:	learn: 1745.5440528	total: 7m 14s	remaining: 1m
878:	learn: 1745.5007801	total: 7m 15s	remaining: 59.9s
879:	learn: 1745.4304879	total: 7m 15s	remaining: 59.4s
880:	learn: 1745.3281287	total: 7m 16s	remaining: 58.9s
881:	learn: 1745.1884648	total: 7m 16s	remaining: 58.4s
882:	learn: 1745.1818480	total: 7m 16s	remaining: 57.9s
883:	learn: 1745.0904748	total: 7m 17s	remaining: 57.4s
884:	learn: 1744.8664506	total: 7m 18s	remaining: 56.9s
885:	learn: 1744.8418856	total: 7m 18s	remaining: 56.4s
886:	learn: 1744.7788986	total: 7m 19s	remaining: 55.9s
887:	learn: 1744.6734402	total: 7m 19s	remaining: 55.4s
888:	learn: 1744.6634627	total: 7m 20s	remaining: 54.9s
889:	learn: 1744.5782562	total: 7m 20s	remaining: 54.5s
890:	learn: 1744.4962700	total: 7m 21s	remaining: 54s
891:	learn: 1744.4391981	total: 7m 21s	remaining: 53.5s
892:	learn: 1744.4178139	total: 7m 22s	remaining: 53s
89

El modelo CatBoost tomó aproximadamente 8 minutos y 18 segundos para completar el entrenamiento. Este tiempo es considerablemente más largo en comparación con modelos más ligeros como LightGBM o RandomForest, pero es típico para modelos complejos que buscan optimizar la precisión mediante técnicas avanzadas como el boosting.

Durante el entrenamiento, el modelo redujo gradualmente el error cuadrático medio (RMSE) de aprendizaje hasta alcanzar un valor de 1737.5936. Esto indica que el modelo está ajustando bien los datos de entrenamiento y aprendiendo las relaciones complejas entre las características y la variable objetivo.

In [52]:
%%time
# Predecimos los precios en el conjunto de prueba
preds_cb = model_cb.predict(test_df[features].fillna(-100))

# Evaluamos el modelo
print(f"""
Catboost performance:
{round(mean_squared_error(test_df[target], preds_cb)**0.5, 4)}
""")


Catboost performance:
1789.4143

CPU times: user 680 ms, sys: 16 µs, total: 680 ms
Wall time: 680 ms


La fase de predicción fue extremadamente rápida, tomando solo 644 milisegundos. Esta eficiencia es una ventaja significativa en aplicaciones donde se requieren predicciones rápidas y en tiempo real.

El RMSE obtenido en la predicción fue de 1789.4143, lo cual es coherente con el rendimiento observado durante el entrenamiento, por lo que es un modelo muy competitivo y muestra una buena capacidad de generalización del modelo.

 ## Modelo CatBoost  (conjunto con variable model).

In [63]:
%%time

# Aseguramos que las características categóricas estén codificadas como cadenas
for col in categorical_features_with_model:
    X_train_with_model[col] = X_train_with_model[col].astype(str)
    X_test_with_model[col] = X_test_with_model[col].astype(str)

# Definimos el modelo CatBoostRegressor
model_cb_with_model = cb.CatBoostRegressor(loss_function='RMSE', iterations=1000)

# Entrenamos el modelo
model_cb_with_model.fit(
    X_train_with_model.fillna('-100'),
    y_train_with_model,
    cat_features=categorical_features_with_model,
    early_stopping_rounds=5
)

Learning rate set to 0.099927
0:	learn: 4228.7932062	total: 786ms	remaining: 13m 4s
1:	learn: 3979.5509232	total: 1.47s	remaining: 12m 11s
2:	learn: 3766.7458114	total: 2.1s	remaining: 11m 38s
3:	learn: 3574.5789184	total: 2.82s	remaining: 11m 42s
4:	learn: 3407.2347158	total: 3.54s	remaining: 11m 44s
5:	learn: 3257.1176971	total: 4.12s	remaining: 11m 22s
6:	learn: 3119.4775514	total: 4.68s	remaining: 11m 4s
7:	learn: 3005.3577854	total: 5.32s	remaining: 11m
8:	learn: 2907.5115378	total: 5.94s	remaining: 10m 53s
9:	learn: 2808.4249434	total: 6.59s	remaining: 10m 52s
10:	learn: 2723.9655788	total: 7.25s	remaining: 10m 51s
11:	learn: 2648.9425691	total: 7.93s	remaining: 10m 53s
12:	learn: 2584.6482808	total: 8.63s	remaining: 10m 55s
13:	learn: 2527.1599228	total: 9.17s	remaining: 10m 46s
14:	learn: 2472.7703628	total: 9.74s	remaining: 10m 39s
15:	learn: 2424.1300878	total: 10.4s	remaining: 10m 39s
16:	learn: 2379.3576421	total: 11s	remaining: 10m 38s
17:	learn: 2343.4116777	total: 11.7s	

148:	learn: 1862.3628624	total: 1m 27s	remaining: 8m 21s
149:	learn: 1861.6947448	total: 1m 28s	remaining: 8m 19s
150:	learn: 1860.2225799	total: 1m 28s	remaining: 8m 19s
151:	learn: 1859.3164150	total: 1m 29s	remaining: 8m 18s
152:	learn: 1858.6695696	total: 1m 29s	remaining: 8m 18s
153:	learn: 1857.9235804	total: 1m 30s	remaining: 8m 17s
154:	learn: 1857.3050541	total: 1m 31s	remaining: 8m 16s
155:	learn: 1856.6274751	total: 1m 31s	remaining: 8m 16s
156:	learn: 1856.0868732	total: 1m 32s	remaining: 8m 15s
157:	learn: 1855.2528596	total: 1m 32s	remaining: 8m 15s
158:	learn: 1854.1571985	total: 1m 33s	remaining: 8m 14s
159:	learn: 1853.2501016	total: 1m 34s	remaining: 8m 15s
160:	learn: 1852.7757728	total: 1m 34s	remaining: 8m 14s
161:	learn: 1851.9782169	total: 1m 35s	remaining: 8m 13s
162:	learn: 1851.4467069	total: 1m 35s	remaining: 8m 12s
163:	learn: 1850.8545845	total: 1m 36s	remaining: 8m 12s
164:	learn: 1850.2281093	total: 1m 37s	remaining: 8m 12s
165:	learn: 1849.5795825	total:

294:	learn: 1790.5882569	total: 2m 54s	remaining: 6m 57s
295:	learn: 1790.4731548	total: 2m 55s	remaining: 6m 57s
296:	learn: 1790.0486995	total: 2m 56s	remaining: 6m 57s
297:	learn: 1789.9175952	total: 2m 56s	remaining: 6m 56s
298:	learn: 1789.6360700	total: 2m 57s	remaining: 6m 56s
299:	learn: 1789.1122144	total: 2m 58s	remaining: 6m 55s
300:	learn: 1788.9244877	total: 2m 58s	remaining: 6m 55s
301:	learn: 1788.5778456	total: 2m 59s	remaining: 6m 54s
302:	learn: 1788.3581148	total: 3m	remaining: 6m 54s
303:	learn: 1788.1826058	total: 3m	remaining: 6m 53s
304:	learn: 1787.6842652	total: 3m 1s	remaining: 6m 53s
305:	learn: 1787.3713450	total: 3m 1s	remaining: 6m 52s
306:	learn: 1786.9652412	total: 3m 2s	remaining: 6m 51s
307:	learn: 1786.6889453	total: 3m 3s	remaining: 6m 51s
308:	learn: 1786.3197307	total: 3m 3s	remaining: 6m 51s
309:	learn: 1786.0628706	total: 3m 4s	remaining: 6m 50s
310:	learn: 1785.8739447	total: 3m 5s	remaining: 6m 50s
311:	learn: 1785.6524609	total: 3m 5s	remainin

439:	learn: 1755.1369540	total: 4m 23s	remaining: 5m 34s
440:	learn: 1754.9604114	total: 4m 23s	remaining: 5m 34s
441:	learn: 1754.6865349	total: 4m 24s	remaining: 5m 33s
442:	learn: 1754.5907692	total: 4m 24s	remaining: 5m 33s
443:	learn: 1754.4093710	total: 4m 25s	remaining: 5m 32s
444:	learn: 1754.1588346	total: 4m 26s	remaining: 5m 31s
445:	learn: 1754.0538438	total: 4m 26s	remaining: 5m 31s
446:	learn: 1753.8791349	total: 4m 27s	remaining: 5m 30s
447:	learn: 1753.6728174	total: 4m 27s	remaining: 5m 30s
448:	learn: 1753.4429892	total: 4m 28s	remaining: 5m 29s
449:	learn: 1753.2604817	total: 4m 29s	remaining: 5m 28s
450:	learn: 1753.0564864	total: 4m 29s	remaining: 5m 28s
451:	learn: 1752.9353129	total: 4m 30s	remaining: 5m 27s
452:	learn: 1752.8762581	total: 4m 30s	remaining: 5m 27s
453:	learn: 1752.7271292	total: 4m 31s	remaining: 5m 26s
454:	learn: 1752.5226324	total: 4m 32s	remaining: 5m 25s
455:	learn: 1752.4086628	total: 4m 32s	remaining: 5m 25s
456:	learn: 1752.1380141	total:

584:	learn: 1731.0830279	total: 5m 48s	remaining: 4m 7s
585:	learn: 1730.8675218	total: 5m 49s	remaining: 4m 6s
586:	learn: 1730.7131584	total: 5m 49s	remaining: 4m 6s
587:	learn: 1730.5537297	total: 5m 50s	remaining: 4m 5s
588:	learn: 1730.5228932	total: 5m 51s	remaining: 4m 5s
589:	learn: 1730.4439254	total: 5m 51s	remaining: 4m 4s
590:	learn: 1730.3338713	total: 5m 52s	remaining: 4m 3s
591:	learn: 1730.2330996	total: 5m 52s	remaining: 4m 3s
592:	learn: 1730.0893384	total: 5m 53s	remaining: 4m 2s
593:	learn: 1729.9694306	total: 5m 54s	remaining: 4m 2s
594:	learn: 1729.9669502	total: 5m 54s	remaining: 4m 1s
595:	learn: 1729.8593242	total: 5m 55s	remaining: 4m
596:	learn: 1729.7490891	total: 5m 55s	remaining: 4m
597:	learn: 1729.6313157	total: 5m 56s	remaining: 3m 59s
598:	learn: 1729.3426366	total: 5m 57s	remaining: 3m 59s
599:	learn: 1729.2127433	total: 5m 57s	remaining: 3m 58s
600:	learn: 1729.0165755	total: 5m 58s	remaining: 3m 58s
601:	learn: 1728.8685667	total: 5m 59s	remaining: 

730:	learn: 1712.1460920	total: 7m 15s	remaining: 2m 40s
731:	learn: 1711.9388191	total: 7m 16s	remaining: 2m 39s
732:	learn: 1711.8352306	total: 7m 16s	remaining: 2m 39s
733:	learn: 1711.8258458	total: 7m 17s	remaining: 2m 38s
734:	learn: 1711.6317236	total: 7m 17s	remaining: 2m 37s
735:	learn: 1711.4556267	total: 7m 18s	remaining: 2m 37s
736:	learn: 1711.4332272	total: 7m 19s	remaining: 2m 36s
737:	learn: 1711.3455115	total: 7m 19s	remaining: 2m 36s
738:	learn: 1711.2542507	total: 7m 20s	remaining: 2m 35s
739:	learn: 1711.1490082	total: 7m 20s	remaining: 2m 34s
740:	learn: 1711.0195789	total: 7m 21s	remaining: 2m 34s
741:	learn: 1710.9042611	total: 7m 22s	remaining: 2m 33s
742:	learn: 1710.8228642	total: 7m 22s	remaining: 2m 33s
743:	learn: 1710.7032242	total: 7m 23s	remaining: 2m 32s
744:	learn: 1710.6294344	total: 7m 24s	remaining: 2m 31s
745:	learn: 1710.2841985	total: 7m 24s	remaining: 2m 31s
746:	learn: 1710.0607142	total: 7m 25s	remaining: 2m 30s
747:	learn: 1709.8391483	total:

875:	learn: 1698.0539933	total: 8m 42s	remaining: 1m 13s
876:	learn: 1697.9218428	total: 8m 42s	remaining: 1m 13s
877:	learn: 1697.7532329	total: 8m 43s	remaining: 1m 12s
878:	learn: 1697.7066244	total: 8m 44s	remaining: 1m 12s
879:	learn: 1697.6445269	total: 8m 44s	remaining: 1m 11s
880:	learn: 1697.4723784	total: 8m 45s	remaining: 1m 10s
881:	learn: 1697.3482575	total: 8m 46s	remaining: 1m 10s
882:	learn: 1697.2069191	total: 8m 46s	remaining: 1m 9s
883:	learn: 1697.0105103	total: 8m 47s	remaining: 1m 9s
884:	learn: 1696.8664397	total: 8m 47s	remaining: 1m 8s
885:	learn: 1696.6806677	total: 8m 48s	remaining: 1m 8s
886:	learn: 1696.5275429	total: 8m 49s	remaining: 1m 7s
887:	learn: 1696.4510840	total: 8m 49s	remaining: 1m 6s
888:	learn: 1696.2917914	total: 8m 50s	remaining: 1m 6s
889:	learn: 1696.2834165	total: 8m 50s	remaining: 1m 5s
890:	learn: 1696.1607955	total: 8m 51s	remaining: 1m 4s
891:	learn: 1696.0860440	total: 8m 51s	remaining: 1m 4s
892:	learn: 1695.9820896	total: 8m 52s	re

In [64]:
%%time

# Predecimos los precios en el conjunto de prueba
preds_cb_with_model = model_cb_with_model.predict(X_test_with_model.fillna('-100'))

# Evaluamos el modelo
print(f"""
Catboost performance with 'Model':
{round(mean_squared_error(y_test_with_model, preds_cb_with_model)**0.5, 4)}
""")


Catboost performance with 'Model':
1744.7205

CPU times: user 946 ms, sys: 0 ns, total: 946 ms
Wall time: 953 ms


La inclusión de la variable 'Model' resultó en una mejora en el rendimiento del modelo. El RMSE se redujo de 1789.4143 a 1744.7205, indicando una mejora en la precisión de las predicciones.

Aunque el rendimiento mejoró, se observó un incremento en el tiempo de ejecución. El tiempo de entrenamiento del modelo aumentó de 8min 17s a 10min 1s, lo que representa aproximadamente un 21% más de tiempo.

El tiempo de predicción también aumentó ligeramente de 680 ms a 953 ms, un incremento del 40%.

La inclusión de la variable 'Model' en el conjunto de datos mejora la precisión del modelo CatBoost, pero a costa de un mayor tiempo de entrenamiento y predicción. 


## Conclusiones

La regresión lineal proporciona una línea base útil y es un buen punto de partida, su desempeño sugiere que se pueden explorar modelos más avanzados para mejorar la precisión de las predicciones. EL RMSE obtenido para el modelo de regresión lineal es de 3138.5415.

Con respecto a la inclusión de la variable 'Model', esto mejoró el rendimiento de todos los modelos probados. Los valores de RMSE se redujeron en todos los casos, lo que indica una mayor precisión en las predicciones al incluir esta variable categórica.

La mejora más significativa se observó en el modelo CatBoost, donde el RMSE se redujo de 1789.4143 a 1744.7205.

El modelo de Bosque Aleatorio con los hiperparámetros optimizados logró un RMSE de 1782.74 con el conjunto de datos con la variable 'Model', lo que representa una mejora significativa en comparación con el modelo de Regresión Lineal. Sin embargo, el costo computacional es considerable, y es importante equilibrar la precisión del modelo con los recursos disponibles.

El modelo LightGBM demostró ser altamente efectivo para predecir el precio de los vehículos en el conjunto de datos utilizado. Incluso con un modelo simple, LightGBM mostró un rendimiento sólido, y la optimización de hiperparámetros permitió mejorar aún más su precisión, llegando a un RMSE de 1770.7392 con el conjunto de datos con la variable 'Model'. 

La capacidad de LightGBM para manejar características categóricas de manera eficiente, junto con su velocidad de entrenamiento, lo convierte en una excelente opción para problemas de regresión con grandes conjuntos de datos.

CatBoost demostró ser un modelo robusto en términos de precisión, reduciendo el error en los datos de entrenamiento y manteniendo un buen rendimiento en los datos de prueba. Además, su eficiencia en la predicción lo hace adecuado para aplicaciones que requieren resultados rápidos, con un RMSE de 1744.7205 para el conjuto de datos con la variable Model. Aunque el tiempo de entrenamiento puede ser más largo en comparación con otros modelos, como LightGBM, el rendimiento obtenido justifica este costo en términos de precisión mejorada y capacidad de manejar datos complejos y ruidosos.

CatBoost es una opción sólida para problemas de regresión donde la precisión es crucial y se necesita una interpretación detallada de las características. Su capacidad para manejar variables categóricas y su robustez ante datos desbalanceados son aspectos adicionales que lo hacen atractivo para aplicaciones del mundo real.


